In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [4]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
#!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00


In [5]:
!ls drive/MyDrive/Colab\ Notebooks/rag_basics_using_llama

chat_doc.txt			       retrieval_basics.ipynb
RAG_basics_using_llama_cpp_cpu.ipynb   zephyr-7b-beta.Q4_0.gguf
RAG_basics_using_llama_cpp_cuda.ipynb


In [6]:
# Paths
document_path = "drive/MyDrive/Colab Notebooks/rag_basics_using_llama/chat_doc.txt"
llama_model_path = "drive/MyDrive/Colab Notebooks/rag_basics_using_llama/zephyr-7b-beta.Q4_0.gguf"

In [10]:
doc_loader = open(document_path)
doc = doc_loader.read()
sentence = doc.split('\n')

In [12]:
sentence[:3]

["A: Hey B, have you been keeping up with the latest advancements in AI? It feels like we're on the verge of something huge.",
 "B: Absolutely! AI is evolving at an insane pace. With models like GPT-4 and Llama, we're already seeing near-human text generation. The big question is: how far are we from AGI?",
 "A: That’s the million-dollar question. Some say AGI could be just a decade away, while others argue that we're still missing key breakthroughs. Do you think scaling up deep learning alone will get us there?"]

In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/72.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [14]:
docs_embed = model.encode(sentence, normalize_embeddings=True)

In [15]:
docs_embed

array([[-0.00151524, -0.02589732, -0.00775498, ..., -0.0201121 ,
        -0.03999557,  0.00779687],
       [-0.00944628, -0.02911086, -0.04643997, ..., -0.0204946 ,
        -0.01445488,  0.02886229],
       [ 0.00994551, -0.01378023, -0.0271623 , ..., -0.03651772,
        -0.03218587, -0.01431904],
       ...,
       [ 0.00490608, -0.03687429, -0.04569393, ..., -0.07674772,
        -0.03045087, -0.00056954],
       [ 0.03925121, -0.00920794, -0.02372728, ..., -0.07524236,
        -0.01496581,  0.00963225],
       [ 0.03712219, -0.06875031, -0.0375578 , ..., -0.09930892,
         0.02145966,  0.005578  ]], dtype=float32)

In [16]:
docs_embed.shape

(11, 768)

In [51]:
#query = 'What areas will AI impact the most?'
query = 'What global challenges can AI solve?'
query_embed = model.encode(query, normalize_embeddings=True)

In [52]:
query_embed.shape

(768,)

In [53]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)

In [54]:
similarities.shape

(11,)

In [55]:
similarities

array([0.44940245, 0.4900781 , 0.606756  , 0.59115756, 0.48998344,
       0.57040787, 0.55287015, 0.4882513 , 0.5213123 , 0.58544815,
       0.45810416], dtype=float32)

In [56]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [57]:
top_3_idx

[2, 3, 9]

In [58]:
most_similar_documents = [sentence[idx] for idx in top_3_idx]

In [59]:
most_similar_documents

["A: That’s the million-dollar question. Some say AGI could be just a decade away, while others argue that we're still missing key breakthroughs. Do you think scaling up deep learning alone will get us there?",
 'B: I doubt it. Scaling helps, but intelligence isn’t just about pattern recognition. We need reasoning, common sense, and the ability to generalize beyond training data. Maybe hybrid models combining symbolic reasoning with deep learning could help.',
 'B: It depends on how we build it. If developed responsibly, AGI could solve global challenges—climate change, disease, healthcare and even space exploration. But if mismanaged, it could be existentially risky.']